# This notebook parses and combines the datasets

In [206]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateparser
import datefinder
import datetime
import os
import json
import re
import difflib
import string

can_province_names = {
      'Alberta':'AB',
      'British Columbia':'BC',
      'Manitoba':'MB',
      'New Brunswick':'NB',
      'Newfoundland and Labrador':'NL',
      'Newfoundland':'NL',
      'Nova Scotia':'NS',
      'Northwest Territories':'NT',
      'Nunavut':'NU',
      'Ontario':'ON',
      'Prince Edward Island':'PE',
      'Quebec':'QC',
      'Saskatchewan':'SK',
      'Yukon':'YT',
    }


can_province_keys = {
      'AB':'Alberta',
      'BC':'British Columbia',
      'MB':'Manitoba',
      'NB':'New Brunswick',
      'NL':'Newfoundland and Labrador',
      'NS':'Nova Scotia',
      'NT':'Northwest Territories',
      'NU':'Nunavut',
      'ON':'Ontario',
      'PE':'Prince Edward Island',
      'QC':'Quebec',
      'SK':'Saskatchewan',
      'YT':'Yukon',
    }


def Assign_Death_Code(df,Keys,Cols,Type,Recat=999,Validate = False,ValCols=None):
    for k in Keys:
        for c in Cols:
            if Validate == True:
                print(df.loc[((df['Death_Category']==Recat)&
                    (df[c].str.contains(k, flags=re.IGNORECASE, regex=True))),
                   ValCols].values)
            else:
                df.loc[((df['Death_Category']==Recat)&
                    (df[c].str.contains(k, flags=re.IGNORECASE, regex=True))),
                   'Death_Category']=Death_Catageory_Code[Type]
            
def FindDates(s):
    flag = 'Parsed'
    if '(' in s and ')' in s:
        d = re.search(r'\((.*?)\)',s).group(1)
        date = dateparser.parse(d)
        if date is None:
            try:
                sp = d.split(',')
                sp1 = sp[-1]
                sp0 = sp[0].split(',')[-1]
                d = sp0.split('-')[-1].split('/')[-1]+sp1#[-1]
#                 d = sp[0].split('-')[-2].split('/')[0]+sp[-1]
                date = dateparser.parse(d)
            except:
                dates = datefinder.find_dates(s)
                D = []
                for d in dates:
                    D.append(d)
                if len(D) > 0:
                    date = D[-1]
                    flag = 'Found'
                else:
                    date = None
                    flag = 'Missing'
                pass
    else:
        dates = datefinder.find_dates(s)
        D = []
        for d in dates:
            D.append(d)
        if len(D) > 0:
            date = D[-1]
            flag = 'Found'
        else:
            date = None
            flag = 'Missing'
    return(date,flag)
            
Death_Catageory_Code = {
    'Shooting':1,
#     'Rubber Bullet':2,
    'Taser':3,
    'Other Weapon':4,
    'Excited Delirium':9,
    'Use of Force':10,
    'Accidental':15,
    'Overdose/Intoxication':25,
    'Vehicle':50,
    'In Custody':75,
    'Suicide':100,
    'Other/Unknown':999
}
Death_Category_SearchKeys = {
    'Shooting':['shoot','shot ','gunshot','rubber bullet','fired'],
#     'Rubber Bullet':[],
    'Taser':['tase','taze','stun gun'],
    'Other Weapon':['baton','club','pepper spray','pepper-spray'],
    'Excited Delirium':['excited delirium'],    
    'Use of Force':['Physical force','restrain','subdue','struggling','struggled','resisted','altercation',
                    'broken rib','collapsed lung',"beat and kick",'dragged','held him down'
#                     'handcuff','apprehended',
#                     'arrested','Anoxic brain injury','interact','trauma','bleeding','blood','strangling',
#                     'head injury','collapsed lung','pushed','fight','resisting','kicked','altercation',
#                     'respiratory failure','injuries'
                   ],
    'Accidental':['Fall','Fell','jump','hypothermia','drowning','suffocat'],
    'Overdose/Intoxication':['Methamphetamine','Cocaine','methadone','alcohol','pills','drugs',
                 'intoxication','overdose','toxicity','blocked with vomit'],
    'Vehicle':['crash','vehicle',"hit by officer's car",'collision'],
    'In Custody':['cell','custody','hospital','Died of an allergy','Found unresponsive','Segregation'],
    'Suicide':['suicide','suicidal','hang','hung','self-inflicted','Incisions of neck'],
    'Other/Unknown':['Unknown']
}

# CBC Deadly Force 2018

### This is the most detailed dataset.  It has:
* location
* race
* gender
* cause of death
* officer information

In [207]:
# incident.csv contains postal code of the incident, province, municipality, along with the date and incident ID
Incident = pd.read_csv(
    'DeadlyForce_2018/incident.csv',
    delimiter = ',',
    header = 0,
    parse_dates=['date'],
    index_col=['id_incident'], 
    encoding='utf-8'
)
# victim.csv contains information on the victms including age, race, etc.
Victim = pd.read_csv(
    'DeadlyForce_2018/victim.csv',
    delimiter = ',',
    header = 0, 
    index_col=['id_incident'], 
    encoding='utf-8'
)
#police.csv contaisn information about the police department and oficer involved
Police = pd.read_csv(
    'DeadlyForce_2018/police.csv',
    delimiter = ',',
    header = 0, 
    index_col=['id_incident'], 
    encoding='utf-8'
)
Join_1 = Incident.join(Victim)
Join_2 = Join_1.join(Police)
PID = Join_2.reset_index().set_index('date',drop = True)
# PID['CBC_2018_id'] = PID['id_incident'].astype(str)+'_CBC_2018'
PID['middle_name']=PID['middle_name'].fillna('')
PID['last_name']=PID['last_name'].fillna('')
PID['name'] = PID['first_name'] +' '+PID['middle_name']+' '+PID['last_name']
PID['name'] = PID['name'].str.replace('  ',' ')
PID['data_source'] = 'Deadly Force (2018)'
# PID['transgender'] = ''  # Only noted if explicitly indicated
# Chevranna was a trans woman killed by hamilton police, summary indicated she was "born male"
# PID.loc[PID['name'] == 'Chevranna Abdi','transgender'] = 'Transgender'
# print(PID.loc[PID['name'] == 'Chevranna Abdi'])
# print(PID['cause_death'].unique())
# print(PID['armed_type'].unique())
PID = PID.rename(columns={'Department':'department',
       'Charges':'charges','Officers Involved':'officers involved'})
# print(PID['substance_abuse'].unique())
# print(PID['mentral_distress_disorder'].unique())
# print(PID.Date)
PID = PID.fillna('')
print(PID.loc[PID['id_victim']=='0417-P1'])

Empty DataFrame
Columns: [id_incident, day_week, prov, city_town, address_intersection, postal_code, location_type, id_victim, first_name, middle_name, last_name, alias_nickname, age, gender, race, ethnic_ancestry, immigrant_refugee_naturalized, armed_type, cause_death, taser_deployed, injured_officer, excited_delirium, mentral_distress_disorder, substance_abuse, summary, department, charge_type, charges, officers involved, name, data_source]
Index: []

[0 rows x 31 columns]


# CBC Deadly Force 2020
### The updated datset is less detailed.


In [208]:
CBC = pd.read_csv('Deadly_Force_2020_Original.csv',
                        parse_dates=['DATE'],
#                         index_col=['VICTIM ID']
                        )
CBC = CBC.rename(columns={'AGE':'age','GENDER':'gender','RACE':'race','SUMMARY':'summary','PROV':'prov',
                          'NAME FIRST':'first_name','NAME MIDDLE':'middle_name','NAME LAST':'last_name',
                          'NICKNAME':'alias_nickname','CAUSE DEATH':'cause_death','ARMED TYPE':'armed_type',
                          'POLICE SERVICE':'department','DATE':'date','VICTIM ID':'id_victim'})
CBC = CBC.drop('PICTURE SOURCE',axis=1)
CBC = CBC.set_index('date')
CBC['middle_name']=CBC['middle_name'].fillna('')
CBC['last_name']=CBC['last_name'].fillna('')
CBC['name'] = CBC['first_name'] +' '+ CBC['middle_name'] +' '+ CBC['last_name']
CBC['name'] = CBC['name'].str.replace('  ',' ')
CBC['age']=CBC['age'].astype(float)
CBC = CBC.fillna('')
CBC['data_source'] = 'Deadly Force (2020)'
# Update records since last 
for key in ['name','race','gender','cause_death','armed_type']:
    UK = PID.loc[((PID['id_victim'].isin(CBC['id_victim']))&(PID[key]=='Unknown')),'id_victim']
    Vals = CBC.loc[((CBC['id_victim'].isin(UK))&(CBC[key]!='Unknown'))]
    if Vals.shape[0]>0:
        print('Updates: ',key,Vals.shape[0])
        PID.loc[((PID['id_victim'].isin(Vals['id_victim']))&(PID[key]=='Unknown')),key]=Vals[key]
        PID.loc[((PID['id_victim'].isin(Vals['id_victim']))&(PID[key]=='Unknown')),'summary']+='; '+Vals['summary']
Missing = CBC.loc[CBC['id_victim'].isin(PID['id_victim'])==False].sort_index()
# print(Missing.columns)
PID = PID.append(Missing)

c = 0
for i, row in PID.iterrows():
    Temp = CBC.loc[((CBC['id_victim']==row['id_victim'])&((CBC['summary']!=row['summary'])))]
    if Temp.shape[0]>0:
        c += 1
        Sum = PID.loc[PID['id_victim']==row['id_victim'],'summary'].values
#         DS = PID.loc[PID['id_victim']==row['id_victim'],'data_source'].values
        try:
            PID.loc[PID['id_victim']==row['id_victim'],'summary'] = Sum +'; '+Temp['summary']
#             PID.loc[PID['id_victim']==row['id_victim'],'data_source'] = DS +'; '+Temp['data_source']
        except:
            pass
#         print(Temp)
# print(c)
# Two records with names are missing summaries
# https://www.baytoday.ca/local-news/no-charges-after-local-man-dies-during-arrest-1393616
PID.loc[PID['name']=='Gordon Dale Couvrette','summary'] = '''No charges will be laid against North Bay police
 officers after a 43-year-old Harris Drive man died after being tasered in his bedroom during an arrest last
 year.'''
# https://www.cbc.ca/news/canada/thunder-bay/police-shooting-lac-seul-first-nation-1.3579724
PID.loc[PID['name']=='Brian Gray','summary'] =  '''The jury at the inquest found that Gray died of\
'gunshot wounds to the torso.' It ruled the death homicide, which in a coroner's inquest means a death as a\
esult of the purposeful actions of another person and does not carry any criminal connotations.'''

# https://toronto.ctvnews.ca/siu-clears-cop-who-shot-and-killed-mississauga-teen-robbery-suspect-1.4160962
PID.loc[PID['name']=='Ozama Shaw','summary'] =  '''The province’s Special Investigations Unit (SIU) has cleared\
a Peel Regional Police officer who shot and killed 15-year-old Ozama Shaw in a Mississauga plaza\
last year, saying the incident appeared on its face to be a case of “shoot or be shot.”'''


# print(PID.loc[PID['summary']=='',['name','department','race','prov','summary','age']])
# print(PID.index)

print(PID.loc[PID['last_name']=='Shaw',['data_source','summary']].values)

Updates:  race 1
Updates:  cause_death 1
Updates:  armed_type 2
[['Deadly Force (2018)'
  'The province’s Special Investigations Unit (SIU) has cleareda Peel Regional Police officer who shot and killed 15-year-old Ozama Shaw in a Mississauga plazalast year, saying the incident appeared on its face to be a case of “shoot or be shot.”']]


# BC Specific Dataset

https://docs.google.com/spreadsheets/d/1aLNSF4Hkk9XdVKeuVU6ZrRO6GtSxT4t8TiMiQ6ptLrY/edit#gid=0

* Data has been pre-processed to remove typos
* I edited/fixed two repeated records maunally

2014.06.15 Jacob George Setah

2014.02.13	Gregory Douglas Lloyd

In [209]:
# Read data and parse dates
GS = pd.read_csv('BC_List_Edited.csv',parse_dates=['Date'])
GS['Circumstances'] = GS[['Circumstances',
                                        'Coroners cause of death',
                                        'Coroners type of death']].fillna('Unknown')
GS = GS.rename(columns={'Name':'name','Age':'age','Sex':'gender','Circumstances':'summary',
                        'Coroners cause of death':'cause_death','City':'city_town',
                          'Agency responsible':'department','Date':'date'})
GS['department'] = GS['department'].str.replace(' / ','; ')
GS['name'] = GS['name'].str.replace('TBD','Unspecified')
GS = GS.set_index('date')

Temp=GS['name'].str.split(' ',expand=True)#.str.len()
Temp['Length'] = GS['name'].str.split(' ').str.len()
# print(Temp[''])
GS['first_name']=Temp[0]
GS['middle_name']=''
GS['last_name']=''
GS['prov']='BC'
GS['id_victim']=[str(i)+'_GS' for i in range(GS.shape[0])]
GS['data_source'] = 'Georgia St (2017)'
for i in range(2,Temp['Length'].max()+1):
#     print(Temp.loc[Temp['Length']==i,[i-1]].shape)
    GS.loc[Temp['Length']==i,'last_name']=Temp.loc[Temp['Length']==i,[i-1]].values
#     print(GS.loc[Temp['Length']==i,'last_name'].shape)
for i in range(3,Temp['Length'].max()+1):
    GS.loc[Temp['Length']==i-1,'middle_name']=Temp.loc[Temp['Length']==i-1,[i-2]].values
# print(GS.columns)
GS = GS.fillna('')
GS['summary']=GS['summary']+'; '+GS['Additional notes']+'; '+GS['Coroners type of death']+\
'; '+GS['Coroners type of death']+'; '+GS['Mental illness / addiction ']
GS = GS.drop(['Additional notes','Coroners type of death','Coroners type of death','Mental illness / addiction '],axis=1)
# print(GS['Additional notes'])
print(GS.columns)

Index(['name', 'age', 'gender', 'city_town', 'department', 'cause_death',
       'summary', 'Inquest date', 'first_name', 'middle_name', 'last_name',
       'prov', 'id_victim', 'data_source'],
      dtype='object')


In [210]:
# print(GS[['first_name','last_name']])
# # print(Temp)
# Temp = GS.loc[GS.first_name=='Duncan','name'].str.split(' ',expand=True)
# Temp['Length'] = GS.loc[GS.first_name=='Duncan','name'].str.split(' ').str.len()
# for i in range(2,Temp['Length'].max()):
#     print(Temp.loc[Temp['Length']==i,[i-1]].values)
print(GS.loc[GS.first_name=='Duncan',['name','first_name','last_name']])#.str.split(' ',expand=True))

                                name first_name last_name
date                                                     
2011-07-16  Duncan Leslie Roy Smears     Duncan    Smears


In [211]:
# Check if there are any "same" names within a 60 day window, allowing for typos.
# Check for missing names within a shorter widow, didnt find any tho
# Add any othere records
Window_named = 60
Window_Unnamed = 5
SimThresh=.75
# c = 0
# GS['Append'] = 1
GS['GS_match'] = 'None'
# PID['CBC_id'] 
for i,row in GS.iterrows():
    if row['name']!='Unknown':
        Match1=(difflib.get_close_matches(row['name'],PID['name'], cutoff=SimThresh))
#         print(Match1)
        Subset = PID.loc[((np.abs(PID.index-i)<datetime.timedelta(Window_named))&(PID['name'].isin(Match1)))]
        if Subset.shape[0]>0:GS.loc[GS.index==i,'GS_match']=Subset['id_victim'].values[0]
        Subset = PID.loc[((np.abs(PID.index-i)<datetime.timedelta(Window_Unnamed))&(PID['name']=='Unknown'))]
        if Subset.shape[0]>0:GS.loc[GS.index==i,'GS_match']=Subset['id_victim'].values[0]
    if row['name']=='Unknown':
        Subset = PID.loc[((np.abs(PID.index-i)<datetime.timedelta(Window_Unnamed))&(PID['name']!='Unknown'))]
        if Subset.shape[0]>0:GS.loc[GS.index==i,'GS_match']=Subset['id_victim'].values[0]
        Subset = PID.loc[((np.abs(PID.index-i)<datetime.timedelta(Window_Unnamed))&(PID['name']=='Unknown'))]
        if Subset.shape[0]>0:GS.loc[GS.index==i,'GS_match']=Subset['id_victim'].values[0]
# print(GS.loc[GS['Match']=='None'].shape)
# print(GS[['GS_match']])
PID = PID.append(GS.loc[GS['GS_match']=='None'])
for i, row in GS.loc[GS['GS_match'].isnull()==False].iterrows():
    Sum = PID.loc[PID['id_victim']==row['GS_match'],'summary']#.values[0]#+'; '+row['summary']
#     print(Sum)
    PID.loc[PID['id_victim']==row['GS_match'],['summary','GS_match']]=Sum,row['id_victim']
    

### laCRAP is a datasest in French 
* It only lists names, ages, dates, and "where" by province, with a few major cities (montreal,toronto,vancouver,ottawa,edmonton) separated out *sometimes
* I've coppied pasted to a text file from this source: https://lacrap.org/liste-des-noms-des-personnes-decedees-aux-mains-de-la-police-au-canada
* I've change the formatting a bit so things worked, and fixed a few typos

In [212]:
Cities = ['Montreal','Montréal','Toronto','Ottawa','Vancouver','Edmonton']
Province = ['Quebec','Quebec','Ontario','Ontario','British Columbia','Alberta']
City_Prov = {k:v for k,v in zip(Cities,Province)}
# laCRAP['Prov']=laCRAP['Prov'].replace(City_Prove)
# City_Police = ['Service de police de la Ville de Montréal','Toronto Police Service','Ottawa Police Service',
#                'Vancouver Police Department','Edmonton Police Service']
# for City,Prov,pol in zip(Cities,Province,City_Police):
#     laCRAP.loc[laCRAP['Prov']==City,['Prov','City','POLICE SERVICE']]=[Prov,City,pol]


#  parses the pre-processed data, and reads the french dates using dateparser
laCRAP = pd.read_csv('laCRAP/Allnames.txt')
laCRAP = laCRAP.dropna().reset_index(drop=True)
laCRAP.Date = laCRAP.Date.apply(lambda x: dateparser.parse(x))
# This cell attributes the location information
Descriptors = (laCRAP.loc[laCRAP.age.str.len()>=5,'age'].str.split(' in ',n=1,expand=True))
laCRAP['prov'] = None
laCRAP['city_town'] = None
laCRAP['department'] = 'Unknown'
laCRAP['summary'] = ''
laCRAP['data_source'] = 'la CRAP (2021)'

for i,l in Descriptors.iterrows():
    try:
        laCRAP.loc[laCRAP.index>i,'prov']=can_province_names[l[1]]
        laCRAP.loc[laCRAP.index>i,'cause_death']=l[0]
    except:
        laCRAP.loc[laCRAP.index>i,'city_town']=l[1]
        laCRAP.loc[laCRAP.index>i,'prov']=can_province_names[City_Prov[l[1]]]
        laCRAP.loc[laCRAP.index>i,'summary']=l[0]
        
laCRAP = laCRAP.loc[laCRAP.Date>'1987-01-01']
Temp=laCRAP['name'].str.split(' ',expand=True)
Temp['Length'] = laCRAP['name'].str.split(' ').str.len()
laCRAP['first_name']=Temp[0]
laCRAP['middle_name']=''
laCRAP['last_name']=''
laCRAP['id_victim']=[str(i)+'_CRAP' for i in range(laCRAP.shape[0])]

for i in range(2,Temp['Length'].max()+1):
    laCRAP.loc[Temp['Length']==i,'last_name']=Temp.loc[Temp['Length']==i,[i-1]].values
for i in range(3,Temp['Length'].max()+1):
    laCRAP.loc[Temp['Length']==i,'middle_name']=Temp.loc[Temp['Length']==i,[i-2]].values

laCRAP = laCRAP.set_index(pd.DatetimeIndex(laCRAP.Date))
laCRAP = laCRAP.drop('Date',axis=1)

In [213]:
# print(laCRAP['Date'])


In [214]:
# print(laCRAP.loc[laCRAP.index>'2021-03-01'])
# print(GS['last_name'].unique())
# for i in range(1,Temp['Length'].max()):

In [215]:
# Check if there are any "same" names within a 60 day window, allowing for typos.
# Check for missing names within a shorter widow, didnt find any tho
# Add any othere records
Window_named = 60
Window_Unnamed = 5
SimThresh=.65
# c = 0
# laCRAP['Append'] = 1
laCRAP['laCRAP_match'] = 'None'
# PID['CBC_id'] 
for i,row in laCRAP.iterrows():    
    if row['name']!='Unknown':
        Match1=(difflib.get_close_matches(row['name'],PID['name'], cutoff=SimThresh))
        Subset = PID.loc[((np.abs(PID.index-i)<datetime.timedelta(Window_named))&(PID['name'].isin(Match1)))]
        if Subset.shape[0]>0:laCRAP.loc[laCRAP.index==i,'laCRAP_match']=Subset['id_victim'].values[0]
        Subset = PID.loc[((np.abs(PID.index-i)<datetime.timedelta(Window_Unnamed))&(PID['name']=='Unknown'))]
        if Subset.shape[0]>0:laCRAP.loc[laCRAP.index==i,'laCRAP_match']=Subset['id_victim'].values[0]
        S = PID.loc[((PID['first_name']==row['first_name'])&(PID['last_name']==row['last_name'])&
                     (np.abs(PID.index-i)<datetime.timedelta(Window_named))
#                                                         (PID['id_victim']==row['id_victim'])
                      )]
#         if S.shape[0]>1:
#             print(S['name','id_victim'])
    else:
        Subset = PID.loc[((np.abs(PID.index-i)<datetime.timedelta(Window_Unnamed))&(PID['name']!='Unknown'))]
        if Subset.shape[0]>0:laCRAP.loc[laCRAP.index==i,'laCRAP_match']=Subset['id_victim'].values[0]
        Subset = PID.loc[((np.abs(PID.index-i)<datetime.timedelta(Window_Unnamed))&(PID['name']=='Unknown'))]
        if Subset.shape[0]>0:laCRAP.loc[laCRAP.index==i,'laCRAP_match']=Subset['id_victim'].values[0]
# print(laCRAP.loc[laCRAP['laCRAP_match']=='None'].shape)
# print(laCRAP.loc[laCRAP['laCRAP_match']!='None'].shape)
# print(laCRAP[['laCRAP_match']])
PID = PID.append(laCRAP.loc[laCRAP['laCRAP_match']=='None'])
for i, row in laCRAP.loc[laCRAP['laCRAP_match'].isnull()==False].iterrows():
    Sum = PID.loc[PID['id_victim']==row['laCRAP_match'],'summary']#.values[0]#+'; '+row['summary']
#     print(Sum)
    PID.loc[PID['id_victim']==row['laCRAP_match'],['summary','laCRAP_match']]=Sum,row['id_victim']
    
print(PID.count()['name'])
# PID = PID[['gender']]
# PID = PID.fillna('Unspecified')
PID = PID.replace({'Unknown':'Unspecified',
                   'Unknown ':'Unspecified'})
PID['race'] = PID['race'].replace({'Arab':'Middle Eastern',
                                  'Caucasian':'White',
                                  '':'Unspecified'})

1465


In [216]:
PID['ds_rank'] = 999
for i,s in enumerate(['Deadly Force (2018)','Deadly Force (2020)','Georgia St (2017)','la CRAP (2021)']):
    PID.loc[PID['data_source']==s,'ds_rank']=i
PID = PID.sort_values(by='ds_rank')

# print(PID.loc[PID['ds_rank']==999])
                     
print(PID.shape)
Named = PID.loc[PID['name']!='Unspecified']
Unspecified = PID.loc[PID['name']=='Unspecified']
# # Named = PID.loc[PID['name']=='Unknown']
# print(PID.groupby('name').count()['id_victim'].sort_values().index)
# # print(PID.loc[PID['name']=='Unknown'])
print(Named.shape)
Named = Named.loc[Named['name'].duplicated(keep=False)==False]
print(Named.shape)

PID = Named.append(Unspecified)
print(PID.shape)

(1465, 35)
(1393, 35)
(1361, 35)
(1433, 35)


In [218]:

PID.index=PID.index.rename('Date')
print(PID.head())
PID.to_csv('CombinedSourceData.csv')

            id_incident day_week prov  city_town         address_intersection  \
Date                                                                            
2012-01-06          1.0      Fri   QC   Montréal  1000 rue De La Gaucheti√®re   
2010-08-25        338.0      Wed   ON  Brantford                91A Sympatica   
2007-11-20        337.0      Tue   ON  Kitchener            1400 Ottawa St. S   
2008-07-31        336.0      Thu   BC     Surrey        106th Ave. & 132 St.    
2008-06-27        335.0      Fri   BC     Surrey        72nd Ave. & 134th St.   

           postal_code location_type id_victim first_name middle_name  ...  \
Date                                                                   ...   
2012-01-06     H3B 4W5         Urban   0001-V1    Farshad              ...   
2010-08-25     N3P 1X6         Urban   0338-V1       Evan              ...   
2007-11-20     N2E 4E2         Urban   0337-V1     Trevor              ...   
2008-07-31     V3T 3V9         Urban   033